In [6]:
import requests
import pendulum
from pathlib import Path
import os
import sys
from tsm import download
import concurrent.futures

In [18]:
diretorio_saida = Path(f'cfs_clima_91_20_daily/')
diretorio_saida.mkdir(exist_ok=True, parents=True)

In [ ]:
####################
# testando arquivos
####################

In [11]:
import xarray as xr

In [10]:
url1=f'https://ftp.emc.ncep.noaa.gov/cmb/sst/oisst_v2/GRIB/oisst.19811101.grb'
nome_arquivo1 = 'oisst.19811101.grb'

url2=f'https://www.ncei.noaa.gov/data/sea-surface-temperature-optimum-interpolation/v2.1/access/avhrr/198111/oisst-avhrr-v02r01.19811101.nc'
nome_arquivo2 = 'oisst-avhrr-v02r01.19811101.nc'

for url, nome_arquivo in zip([url1,url2], [nome_arquivo1, nome_arquivo2]):
    resp=requests.get(url)

    if resp.status_code == 200:
        resp=requests.get(url).content
        diretorio = Path(diretorio_saida, nome_arquivo)

        with open(diretorio, "wb") as arquivo_:
            arquivo_.write(resp)
            print(f"{nome_arquivo} [ok]")
            mensagem_de_sucesso = True

oisst.19811101.grb [ok]
oisst-avhrr-v02r01.19811101.nc [ok]


In [15]:
ds1 = xr.open_dataset(f'cfs_clima_91_20_daily/{nome_arquivo1}') 
ds2 = xr.open_dataset(f'cfs_clima_91_20_daily/{nome_arquivo2}') 

In [16]:
ds1

<xarray.Dataset>
Dimensions:     (latitude: 180, longitude: 360)
Coordinates:
    time        datetime64[ns] ...
    step        timedelta64[ns] ...
    surface     float64 ...
  * latitude    (latitude) float64 89.5 88.5 87.5 86.5 ... -87.5 -88.5 -89.5
  * longitude   (longitude) float64 -0.5 0.5 1.5 2.5 ... 355.5 356.5 357.5 358.5
    valid_time  datetime64[ns] ...
Data variables:
    t           (latitude, longitude) float32 ...
    atmw        (latitude, longitude) float32 ...
    p3091       (latitude, longitude) float32 ...
Attributes:
    GRIB_edition:            1
    GRIB_centre:             kwbc
    GRIB_centreDescription:  US National Weather Service - NCEP
    GRIB_subCentre:          4
    Conventions:             CF-1.7
    institution:             US National Weather Service - NCEP
    history:                 2022-09-28T16:52 GRIB to CDM+CF via cfgrib-0.9.1...

In [22]:
ds2

<xarray.Dataset>
Dimensions:  (time: 1, zlev: 1, lat: 720, lon: 1440)
Coordinates:
  * lat      (lat) float32 -89.88 -89.62 -89.38 -89.12 ... 89.38 89.62 89.88
  * lon      (lon) float32 0.125 0.375 0.625 0.875 ... 359.1 359.4 359.6 359.9
  * time     (time) datetime64[ns] 1981-11-01T12:00:00
  * zlev     (zlev) float32 0.0
Data variables:
    anom     (time, zlev, lat, lon) float32 ...
    err      (time, zlev, lat, lon) float32 ...
    ice      (time, zlev, lat, lon) float32 ...
    sst      (time, zlev, lat, lon) float32 ...
Attributes: (12/37)
    title:                      NOAA/NCEI 1/4 Degree Daily Optimum Interpolat...
    source:                     ICOADS, NCEP_GTS, GSFC_ICE, NCEP_ICE, Pathfin...
    id:                         oisst-avhrr-v02r01.19811101.nc
    naming_authority:           gov.noaa.ncei
    summary:                    NOAAs 1/4-degree Daily Optimum Interpolation ...
    cdm_data_type:              Grid
    ...                         ...
    metadata_link:              https://doi.org/10.25921/RE9P-PT57
    ncei_template_version:      NCEI_NetCDF_Grid_Template_v2.0
    comment:                    Data was converted from NetCDF-3 to NetCDF-4 ...
    sensor:                     Thermometer, AVHRR
    Conventions:                CF-1.6, ACDD-1.3
    references:                 Reynolds, et al.(2007) Daily High-Resolution-...

In [ ]:
##################
# fim teste
#################

In [19]:
lista = list()

inicio = pendulum.datetime(1991,1,1)
fim = pendulum.datetime(2020, 12, 31)
periodo = pendulum.period(inicio, fim)

for dt in periodo.range('days'):
    data_string = dt.format('DD-MM-YYYY')
    lista.append(data_string)
    #download(data_string=data_string)

In [ ]:
#######################################################################################################
# ATENÇÃO : ESTE BLOCO EXECUTARÁ O DOWNLOAD DE UMA GRANDE QUANTIDADE DE ARQUIVOS QUE JÁ FOI REALIZADA #
# RODÁ-LO DE NOVO NÃO VAI EXPLODIR O PC, MAS VAI GASTAR 10 MINUTOS DA SUA VIDA PARA NADA              #
#######################################################################################################

print(f"inicio: {pendulum.now().format('HH:mm:ss')}")

with concurrent.futures.ThreadPoolExecutor(max_workers=100) as executor:
        requisicoes = executor.map(download, lista)

print(f"fim: {pendulum.now().format('HH:mm:ss')}")

In [24]:
dataset = xr.open_mfdataset(f'{diretorio_saida}/*.nc', concat_dim="time", combine="nested",
                data_vars='minimal', coords='minimal', compat='override',parallel=True)

In [25]:
dataset

,Array,Chunk
Bytes,42.32 GiB,3.96 MiB
Shape,"(10958, 1, 720, 1440)","(1, 1, 720, 1440)"
Count,21917 Graph Layers,10958 Chunks
Type,float32,numpy.ndarray
,Array,Chunk
Bytes,42.32 GiB,3.96 MiB
Shape,"(10958, 1, 720, 1440)","(1, 1, 720, 1440)"
Count,21917 Graph Layers,10958 Chunks
Type,float32,numpy.ndarray
,Array,Chunk


In [29]:
# para criar climatologia
anom_tsm = dataset.anom.groupby('time.month').mean()
anom_tsm = anom_tsm.isel(zlev=0)

In [35]:
anom_tsm

<xarray.DataArray 'anom' (month: 12, lat: 720, lon: 1440)>
dask.array<getitem, shape=(12, 720, 1440), dtype=float32, chunksize=(1, 720, 1440), chunktype=numpy.ndarray>
Coordinates:
  * lat      (lat) float32 -89.88 -89.62 -89.38 -89.12 ... 89.38 89.62 89.88
  * lon      (lon) float32 0.125 0.375 0.625 0.875 ... 359.1 359.4 359.6 359.9
    zlev     float32 0.0
  * month    (month) int64 1 2 3 4 5 6 7 8 9 10 11 12

In [81]:
import pandas as pd
from regioes_indices import regioes
import xarray as xr
import pendulum

import recorta_dado
from utils import padronizar_longitude
from utils import delimitar_regiao
from utils import converter_para_celsius

In [64]:
dicionario = {
    'mes': [],
    "TSM Niño 1+2" : [],
    "TSM Niño 3" : [],
    "TSM Niño 3.4" : [],
    "TSM Niño 4" : [],
    "TSM CBM" : [],
    "TSM TNA" : [],
    "TSM TSA" : [],
}

for mes in range(1,12+1):
    tsm = anom_tsm.sel(month=mes)
    dicionario['mes'].append(mes)
    
    for indice in regioes:
        
        tsm = padronizar_longitude(tsm, xdim='lon')
        tsm_ = tsm.compute()
        valor = delimitar_regiao(obj=tsm_, indice=indice)
        
        valor = round(float(valor.values), 2)

        dicionario[indice.nome].append(valor)

        print(f'O valor da região {indice.nome} no mês {mes} é : {valor}')


KeyboardInterrupt



In [82]:
dicionario = {
    'mes': [],
    "TSM Niño 1+2" : [],
    "TSM Niño 3" : [],
    "TSM Niño 3.4" : [],
    "TSM Niño 4" : [],
    "TSM CBM" : [],
    "TSM TNA" : [],
    "TSM TSA" : [],
}
for mes in range(1,12+1):
    tsm = anom_tsm.sel(month=mes)
    dicionario['mes'].append(mes)
    
    for indice in regioes:
    
        tsm_nino = recorta_dado.main(tsm,
            contorno_tipo='lat_lon',
            tipo='nc',
            latitude=[ indice.latitude[0], indice.latitude[1] ], 
            longitude=[ indice.longitude[0], indice.longitude[1] ]
            )
        
        valor = tsm_nino.mean(dim=["lat", "lon"]) 
        valor = round(float(valor.values), 2)

        dicionario[indice.nome].append(valor)

        print(f'O valor da região {indice.nome} no mês {mes} é : {valor}')

KeyError: "'longitude' is not a valid dimension or coordinate"

In [65]:
def delimitar_regiao2(obj: xr.DataArray, indice) -> xr.DataArray:
    """
    Delimitação de regiões do oceano Atlântico e seu valor médio na região.
    Para um dado índice.
    Parameters
    ----------
    obj : xr.DataArray
        Dados da rodada do modelo
    indice : Any
        Informações sobre o índice vindos do módulo schema.nino
    Returns
    -------
    xr.DataArray
        Série com as temperaturas previstas.
    """
    rec = obj.sel(
        lat=slice(indice.latitude[0], indice.latitude[1]),
        lon=slice(indice.longitude[0], indice.longitude[1]),
    ).mean(dim=["lat", "lon"])


    valor = rec.compute()

    return rec

In [74]:
regioes[2]

Indices(id=34, latitude=(5, -5), longitude=(-170, -120), nome='TSM Niño 3.4')

In [75]:
valor2 = delimitar_regiao2(obj=tsm_, indice=regioes[2])

In [42]:
valor = delimitar_regiao(obj=tsm, indice=indice)

In [76]:
valor2

<xarray.DataArray 'anom' ()>
array(nan, dtype=float32)
Coordinates:
    zlev     float32 0.0
    month    int64 1

In [70]:
tsm_ = tsm.compute()

In [77]:
tsm_

<xarray.DataArray 'anom' (lat: 720, lon: 1440)>
array([[       nan,        nan,        nan, ...,        nan,        nan,
               nan],
       [       nan,        nan,        nan, ...,        nan,        nan,
               nan],
       [       nan,        nan,        nan, ...,        nan,        nan,
               nan],
       ...,
       [0.0644301 , 0.06321505, 0.06733333, ..., 0.06224731, 0.06378495,
        0.06507526],
       [0.06054838, 0.0588172 , 0.07258064, ..., 0.061     , 0.06295698,
        0.06370967],
       [0.05286021, 0.06195698, 0.10655913, ..., 0.05113978, 0.05156989,
        0.0520215 ]], dtype=float32)
Coordinates:
  * lat      (lat) float32 -89.88 -89.62 -89.38 -89.12 ... 89.38 89.62 89.88
  * lon      (lon) float32 -179.9 -179.6 -179.4 -179.1 ... 179.4 179.6 179.9
    zlev     float32 0.0
    month    int64 1